In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np
from transformers import EarlyStoppingCallback, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

In [ ]:
# Tạo tokenizer và model từ RoBERTa
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/val_data.csv')

train_texts = train_df['content']
train_labels = train_df['label']
val_texts = val_df['content']
val_labels = val_df['label']

In [ ]:
# Tokenize dữ liệu với padding
def tokenize_and_pad(texts, tokenizer, max_length=256):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',  # Thêm padding để tất cả các văn bản có cùng chiều dài
        max_length=max_length
    )
    return encodings

train_encodings = tokenize_and_pad(train_texts.tolist(), tokenizer, max_length=256)
val_encodings = tokenize_and_pad(val_texts.tolist(), tokenizer, max_length=256)

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).squeeze()  # Đảm bảo nhãn có kích thước đúng
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = FakeNewsDataset(train_encodings, train_labels.tolist())
val_dataset = FakeNewsDataset(val_encodings, val_labels.tolist())

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
# Thiết lập các tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Huấn luyện mô hình
trainer.train()

Step,Training Loss
10,0.709200
20,0.693600
30,0.691000
40,0.695500
50,0.689100
60,0.700100
70,0.688700
80,0.696600
90,0.686700
100,0.689300


Step,Training Loss
10,0.709200
20,0.693600
30,0.691000
40,0.695500
50,0.689100
60,0.700100
70,0.688700
80,0.696600
90,0.686700
100,0.689300


TrainOutput(global_step=429, training_loss=0.6354851567106091, metrics={'train_runtime': 9925.1397, 'train_samples_per_second': 0.345, 'train_steps_per_second': 0.043, 'total_flos': 450709237831680.0, 'train_loss': 0.6354851567106091, 'epoch': 3.0})

In [ ]:
# Dự đoán nhãn cho tập kiểm tra
predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)

In [ ]:
# Tính các chỉ số
accuracy = accuracy_score(val_labels, pred_labels)
precision = precision_score(val_labels, pred_labels, pos_label=1)
recall = recall_score(val_labels, pred_labels, pos_label=1)
f1 = f1_score(val_labels, pred_labels, pos_label=1)
auc = roc_auc_score(val_labels, predictions.predictions[:, 1])
print(f"Accuracy: {accuracy:.6f}")
print(f"Precision: {precision:.6f}")
print(f"Recall: {recall:.6f}")
print(f"F1 Score: {f1:.6f}")
print(f'AUC: {auc:.6f}')

Accuracy: 0.716783
Precision: 0.700599
Recall: 0.790541
F1 Score: 0.742857
AUC: 0.799060


In [ ]:
output_dir = '/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Report1/RoBERTa_fakenews/tokenizer.json')

In [ ]:
# Thiết lập các tham số huấn luyện (có dừng sớm)
training_args_with_early_stop = TrainingArguments(
    output_dir='./results_with_early_stop',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs_with_early_stop',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
)

# Khởi tạo Trainer (có dừng sớm)
trainer_with_early_stop = Trainer(
    model=model,
    args=training_args_with_early_stop,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer_with_early_stop.train()

Step,Training Loss,Validation Loss
50,0.703500,0.691389
100,0.703500,0.679010
150,0.606100,0.598801
200,0.606500,0.666796
250,0.462500,0.522592
300,0.631800,0.756723
350,0.655100,0.717477
400,0.679900,0.560489


TrainOutput(global_step=400, training_loss=0.6325934028625488, metrics={'train_runtime': 7775.9964, 'train_samples_per_second': 0.723, 'train_steps_per_second': 0.091, 'total_flos': 419925244354560.0, 'train_loss': 0.6325934028625488, 'epoch': 2.8368794326241136})

In [ ]:
# Dự đoán nhãn cho tập kiểm tra
predictions2 = trainer_with_early_stop.predict(val_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels2 = np.argmax(predictions2.predictions, axis=1)

In [ ]:
# In ra một số dự đoán để kiểm tra
print(pred_labels2[:10])  # In ra 10 dự đoán đầu tiên

[0 0 0 1 1 0 1 1 1 0]


In [ ]:
print(pred_labels2[:])  # In ra dự đoán

[0 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 1 0 1 1 1
 0 1 1 1 0 1 1 1 1 0 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1
 1 1 0 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0
 0 0 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 1 0 1 0 1
 0 0 1 1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 0
 0 0 1 1 0 1 0 1 1 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1
 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1]


In [ ]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=0)
recall2 = recall_score(val_labels, pred_labels2, pos_label=0)
f12 = f1_score(val_labels, pred_labels2, pos_label=0)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.741135
Precision: 0.844660
Recall: 0.604167
F1 Score: 0.704453
AUC: 0.834541
